<a href="https://colab.research.google.com/github/daicamino/lecture-ai-engineering/blob/master/PJT04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# JPT04 講義資料の質疑応答のまとめ機能の作成

神野大輔

##はじめに

　本プログラムは、『講義内容質問フォーム（回答）』の質問を要約したうえで並び替え、講師が類似した質問をまとめて回答することを支援する質問要約ツールである。

　生成モデルは、本講座の第３回演習のプログラムを引用しLlama3を利用している。また、それ以外のコードのドラフトはGeminiにより生成した。

　「質問」または「要約」をもとに類似度によるグループ分けを試みたが、分類の精度が上げられなかった。また、要約をソートすることでもある程度視認性が改善できるため、質問の内容による分類は不採用とした。

　加えて、処理中にも新しい質問が追加される可能性があるため、要約のソートは手動で行うこととする。

　なお、モデルのダウンロードには時間を要するため、本プログラムの利用時には初期設定までは事前に完了していることが望ましい。



##取扱方法

1.   hugging faceのトークンを準備ください。
2.   『講義内容質問フォーム（回答）』のB列に「要約」を追加ください。もしも、プログラム実行時に該当の列がないばあいは、「要約」列が最後列に自動で追加されます。
3.   『講義内容質問フォーム（回答）』を、Google スプレッドシート形式で保存してください。
4.   このノートブックを「講義質問フォーム」と同じフォルダに格納
してください。
5.   処理完了後、『講義内容質問フォーム（回答）』を開き、手動にて「要約」列でソートしてください。

# 初期設定

##環境設定
ご自身の環境に合わせて変更ください。

In [1]:
# 作業ディレクトリとQAファイル名の指定
work_dir = '/content/drive/MyDrive/Colab Notebooks/lecture-ai-engineering/PJT04/'
QAsheet = '講義内容質問フォーム（回答）'

# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import random
random.seed(0)

## Goodle Driveへの接続と各種認証

In [2]:
# Goodle Driveへの接続
from google.colab import drive
drive.mount('/content/drive')

# colab Notebookの認証
from google.colab import auth
auth.authenticate_user()

# gspreadの認証
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

Mounted at /content/drive


## LLMモデルの読み込み

　本講義の第３回演習のプログラムより引用。


In [3]:
# HuggingFace Login
from huggingface_hub import notebook_login
notebook_login()

In [4]:
# モデル(Llama3)の読み込み

!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 100.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 115.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.14.0
    Uninstalling typing_extensions-4.14.0:
      Successfully uninstalled typing_extensions-4.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

#処理

##GSpreadの読み込み

In [5]:
import pandas as pd

worksheet = gc.open(QAsheet).sheet1
rows = worksheet.get_all_values()
qa_data = pd.DataFrame(rows[1:], columns=rows[0])

# 「要約」列が存在しない場合は追加
if '要約' not in qa_data.columns:
    qa_data['要約'] = ''
    # スプレッドシートにも列を追加 (ヘッダー行)
    worksheet.insert_cols([['要約']], col=len(rows[0]) + 1)

# prompt: qa_dataの内容を表示
qa_data.head()

,タイムスタンプ,要約,質問,氏名（任意）,スライド番号（任意）
0,,,データパイプラインの概念があまり理解できていないのですが、実際に分かりやすいアプリケーション...,,
1,,,Googlecolab はどんなアカウントを使ってもいいですか,,
2,,,既存論文を再現するとのことでしたが、公開されているプログラムを自身の環境でコピーして、一部は...,,
3,,,AIを使ったプロダクトのライフサイクル (プロとタイミング、実開発、検証、再開発などの流れ)...,,
4,,,FastAPIの役割と存在意義が理解できなかったです。\r\n「型ヒントとPydanticで...,,


##質問の要約

In [6]:
# prompt: もしも'要約'が空白のとき、各行の質問を要約して、ワークシートの'要約'列に挿入する。

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# 各行の「要約」列が空白かどうかを確認し、空白の場合は要約を生成して挿入
for index, row in qa_data.iterrows():
    if row['要約'] == '':
        question = row['質問']
        messages = [
            {"role": "system", "content": "コンテンツの件名を30文字以内で生成する。コンテンツに記載されていないことは表示しない。日本語で回答する。"},
            {"role": "user", "content": f"コンテンツ：{question}"},
        ]
        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        summary_output = model.generate(
            input_ids,
            max_new_tokens=25, # 要約のトークン数
            eos_token_id=terminators,
            do_sample=False,
        )

        summary_tokens = summary_output[0][input_ids.shape[-1]:]
        summary_str = tokenizer.decode(summary_tokens, skip_special_tokens=True)
        summary_str = summary_str.replace('タイトル：', '').replace('件名：', '').replace('「', '').replace('」', '')

        # DataFrameの「要約」列を更新
        qa_data.at[index, '要約'] = summary_str

        # スプレッドシートの該当セルを更新
        # ヘッダー行があるため、DataFrameのインデックスに2を加える (0始まりのDataFrameインデックス + 1 (ヘッダー) + 1 (スプレッドシートは1始まり))
        # 「要約」列のインデックスを取得
        summary_col_index = qa_data.columns.get_loc('要約') + 1 # gspreadは1始まり

        worksheet.update_cell(index + 2, summary_col_index, summary_str)

print("要約の更新が完了しました。")
print(qa_data[['質問', '要約']].head()) # 更新されたデータの確認

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for op

要約の更新が完了しました。
                                                  質問  \
0  データパイプラインの概念があまり理解できていないのですが、実際に分かりやすいアプリケーション...   
1                    Googlecolab はどんなアカウントを使ってもいいですか   
2  既存論文を再現するとのことでしたが、公開されているプログラムを自身の環境でコピーして、一部は...   
3  AIを使ったプロダクトのライフサイクル (プロとタイミング、実開発、検証、再開発などの流れ)...   
4  FastAPIの役割と存在意義が理解できなかったです。\r\n「型ヒントとPydanticで...   

                                                  要約  
0            データパイプラインの実際のアプリケーション：E-commerceの商品情報集積  
1  "Google Colab：無料のJupyterノートブック環境、Googleアカウントでア...  
2                 既存論文を再現するという意味は、既に公開されているプログラムを自身の  
3                AIプロダクトライフサイクルマネジメント：AI技術の導入、開発、テスト  
4          FastAPIの理解を深める：型ヒントとPydanticによる自動バリデーションの  
